In [1]:
#Import packages
import pandas as pd
import numpy as np
import glob
import re
import string
import codecs

In [4]:
li = []

for file in glob.glob('./Parliament_Qs/rajyasabha_questions_and_answers_*.csv'):
    data = pd.read_csv(file, encoding='ISO-8859-1')
    li.append(data)
    
parliament_qs = pd.concat(li, axis = 0, ignore_index = True)

/Users/murielpokol/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


In [7]:
#Code borrowed and adapted from George Chen, Carnegie Mellon University#
#Define function to remove punctuation and whitespace, and lowercase all text
def makeWordList(str_object):
    
    corpus_text = str(str_object)
    
    for c in string.punctuation:
        corpus_text = corpus_text.replace(c, "")  # -- (1)
    
    text = re.sub(r'\S*\d\S*','',corpus_text) # -- (2)
    text = re.sub(r'[^\w\s]','',text)         # -- (3)
    
    text = text.lower().split()           # -- (4)         
    
    li = []
    for token in text:
        li.append(token)

    return " ".join(li)

In [8]:
#Process the questions
processed_questions = []

for str_object in list(parliament_qs["question_description"]):
    processed_questions.append(makeWordList(str_object))

In [9]:
#Use CountVectorizer to transform parliamentary questions
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=200, stop_words="english", max_df=0.8)
questions_fit = vectorizer.fit(processed_questions)
X_questions = vectorizer.fit_transform(processed_questions).toarray()

In [10]:
#!Time-consuming!#
#Create topics using LDA
num_topics = 30

from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=num_topics, learning_method='online', random_state=0)
lda.fit(X_questions)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=30, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [15]:
#Display top 20 words from each topic
words = list(questions_fit.vocabulary_)
topic_word_distributions = np.array([row / row.sum() for row in lda.components_])
num_top_words = 20

print('Displaying the top 20 words per topic and their probabilities within the topic...')
print()

for topic_idx in range(num_topics):
    print('[Topic ', topic_idx, ']', sep='')
    sort_indices = np.argsort(-topic_word_distributions[topic_idx])
    for rank in range(num_top_words):
        word_idx = sort_indices[rank]
        print(words[word_idx], ':', topic_word_distributions[topic_idx, word_idx])
    print()

Displaying the top 20 words per topic and their probabilities within the topic...

[Topic 0]
centres : 0.06125583934276024
compensation : 0.04931367838211531
modernisation : 0.04517677473651625
achieving : 0.04336182347580504
collected : 0.042224825513318937
villages : 0.042146554853484196
answer : 0.03638924412105475
menace : 0.03505791366874948
published : 0.028148082741535452
life : 0.02728750277305654
stop : 0.026518378335301775
natural : 0.024814826269095407
selection : 0.023946566520880824
filed : 0.02302117478059483
relating : 0.020768956555096155
record : 0.02071579593335945
introduce : 0.020692942275840853
utilised : 0.020567920688699125
college : 0.018840926150507
thereofe : 0.016825301688058222

[Topic 1]
gap : 0.08628711357140811
energy : 0.0681506598815247
experts : 0.0578801522387212
house : 0.050718452864265864
infrastructure : 0.046534995034547166
approved : 0.03667078115666605
discussion : 0.03618805559181497
generating : 0.027133924093691975
permanent : 0.026844146380